In [1]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from citipy import citipy
from config import weather_api_key
from datetime import datetime
from scipy.stats import linregress

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90.0, 90.0, size=1500)
lngs = np.random.uniform(-90.0, 90.0, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
coordinates = list(lat_lngs)

In [4]:
# Create a list of cities from the list of latitude & longitude
cities = []

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # If the city does not exist in the list, add it to the cities list
    if city not in cities:
        cities.append(city)
len(cities)

602

In [5]:
# URL for Weather Map API Call
OpenWeather_url = "https://api.openweathermap.org/data/2.5/weather?"

In [6]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city name
    query_url = OpenWeather_url + "units=imperial" + "&appid=" + weather_api_key + "&q=" + city

    # Log the URL, record and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to record_count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(query_url).json()
        
        # Retrieve required data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_maxtemp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        city_current_description = city_weather["weather"][0]["description"]
        
        # Capturing amount of rain
        try:
            city_rain3hr = city_weather['rain']['3h']
        except KeyError:
            try:
                city_rain3hr = city_weather['rain']['1h']
            except KeyError:
                city_rain3hr = 0
        
        # Capturing amount of snow
        try:
            city_snow3hr = city_weather['snow']['3h']
        except KeyError:
            try:
                city_snow3hr = city_weather['snow']['1h']
            except KeyError:
                city_snow3hr = 0
        
        
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append data into city_data list
        city_data.append({"City":city.title(),
                         "Lat":city_lat,
                         "Lng":city_lng,
                         "Max Temp":city_maxtemp,
                         "Humidity":city_humidity,
                         "Cloudiness":city_clouds,
                         "Wind Speed":city_wind,
                         "Country":city_country,
                         "Date":city_date,
                         "Current Description":city_current_description,
                         "Rain inches (last 3hrs)":city_rain3hr,
                         "Snow inches (last 3hrs)":city_snow3hr})
    
    # In case of error, skip the current city
    except:
        print("City Not Found. Skipping...")
        pass

# Completion of Data Retrieval
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | hermanus
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | yar-sale
Processing Record 5 of Set 1 | torbay
Processing Record 6 of Set 1 | karak
Processing Record 7 of Set 1 | ozgon
City Not Found. Skipping...
Processing Record 8 of Set 1 | havre-saint-pierre
Processing Record 9 of Set 1 | port alfred
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | jamestown
Processing Record 12 of Set 1 | kruisfontein
Processing Record 13 of Set 1 | illoqqortoormiut
City Not Found. Skipping...
Processing Record 14 of Set 1 | dikson
Processing Record 15 of Set 1 | bredasdorp
Processing Record 16 of Set 1 | codrington
Processing Record 17 of Set 1 | grand gaube
Processing Record 18 of Set 1 | vila franca do campo
Processing Record 19 of Set 1 | itupeva
Processing Record 20 of Set 1 | melilla
Processing Record 21 of Set 1 | saint ant

In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df = city_data_df[['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Current Description','Rain inches (last 3hrs)','Snow inches (last 3hrs)']]

In [12]:
city_data_df.loc[(city_data_df['Rain inches (last 3hrs)'] > 0) | (city_data_df['Snow inches (last 3hrs)'] > 0)].count()

City                       80
Country                    80
Date                       80
Lat                        80
Lng                        80
Max Temp                   80
Humidity                   80
Cloudiness                 80
Wind Speed                 80
Current Description        80
Rain inches (last 3hrs)    80
Snow inches (last 3hrs)    80
dtype: int64

In [13]:
# Create the output file (csv)
output_data_file = "weather_data/WeatherPy_challenge.csv"
# Export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
snow_rain_city = city_data_df.loc[(city_data_df['Rain inches (last 3hrs)'] > 0) | (city_data_df['Snow inches (last 3hrs)'] > 0)]['City'].count()
rain_city = city_data_df.loc[(city_data_df['Rain inches (last 3hrs)'] > 0)]['City'].count()
snow_city = city_data_df.loc[(city_data_df['Snow inches (last 3hrs)'] > 0)]['City'].count()
print(f"{snow_rain_city} cities have recorded rainfall or snow (rain:{rain_city}, snow:{snow_city})")

80 cities have recorded rainfall or snow (rain:60, snow:20)
